In [ ]:
! pip install --upgrade codetiming numpy pandas census us mechanicalsoup geopandas pandas_bokeh torch
get_ipython().kernel.do_shutdown(True)

In [18]:
mount_path = '/content/drive'
repo_path = f'{mount_path}/MyDrive/gerrymandering/2022-10/voting_predictor'
from google.colab import drive
drive.mount(mount_path)
%cd {repo_path}
%load_ext google.colab.data_table
%load_ext autoreload
%autoreload
# from acs5 import *
from model import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/gerrymandering/2022-10/voting_predictor
The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[autoreload of model failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/content/drive/My Drive/gerrymandering/2022-10/voting_predictor/model.py", line 2, in <module>
    device = "cuda" if torch.cuda.is_available() else "cpu"
NameError: name 'torch' is not defined
]


In [19]:
pq = features(2020)

In [2]:
class Feedforward(torch.nn.Module):
    def __init__(self, input_size, hidden_size, activation):
        super(Feedforward, self).__init__()
        activation = listify(activation)
        hidden_size = listify(hidden_size)
        q = hidden_size[0]
        L = [torch.nn.Linear(input_size, q)]
        p = q
        for f, q in zip(activation, hidden_size):
            L.append(f())
            L.append(torch.nn.Linear(p, q))
            p = q
        self.model = torch.nn.Sequential(*L)

    def forward(self, x, w):
        self.prop = self.model(x)
        self.pred = ((self.prop * w) @ b).squeeze()
        return self.pred


print(f"Using {device} device")

model = Feedforward(
    input_size = x_train.shape[1],
    hidden_size = (50, 6),
    activation = (torch.nn.ReLU, torch.nn.Sigmoid)
).to(device)
loss_fcn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

model.eval()
print(f'Test loss before training {loss_fcn(model(x_test, w_test), y_test)}')

model.train()
epoch = 1000
for epoch in range(epoch):
    optimizer.zero_grad()             # Clear gradient
    y_pred = model(x_train, w_train)  # Forward propagation
    loss = loss_fcn(y_pred, y_train)  # Compute Loss
    if epoch % 100 == 0:
        print(f'Epoch {epoch}: train loss {loss}')
    loss.backward()                   # Backward propagation
    optimizer.step()                  # Learn

model.eval()
print(f'Test loss after training {loss_fcn(model(x_test, w_test), y_test)}')

,vtd2020,year,fips,county,election,office,party,name,incumbent,campaign,votes,all_tot_pop
index,,,,,,,,,,,,
0,4832100001C,2016,48321,Matagorda,general,CCA2,L,Ash,False,2016_general_CCA2_Ash_L,9,1581
1,48207000005,2016,48207,Haskell,general,CCA2,L,Ash,False,2016_general_CCA2_Ash_L,7,584
2,48497000020,2016,48497,Wise,general,CCA2,L,Ash,False,2016_general_CCA2_Ash_L,22,1775
3,48149000022,2016,48149,Fayette,general,CCA2,L,Ash,False,2016_general_CCA2_Ash_L,3,249
4,48401000416,2016,48401,Rusk,general,CCA2,L,Ash,False,2016_general_CCA2_Ash_L,11,1676
5,4803300003A,2016,48033,Borden,general,CCA2,L,Ash,False,2016_general_CCA2_Ash_L,1,54
6,48139000147,2016,48139,Ellis,general,CCA2,L,Ash,False,2016_general_CCA2_Ash_L,102,6016
7,48299000109,2016,48299,Llano,general,CCA2,L,Ash,False,2016_general_CCA2_Ash_L,41,3328
8,48097000001,2016,48097,Cooke,general,CCA2,L,Ash,False,2016_general_CCA2_Ash_L,31,3073
